In [1]:
from tvDatafeed import TvDatafeed, Interval
import mysql.connector
import pandas as pd

In [2]:
def query(query, host="ideatrade.serveftp.net", port=51410, user="Wasan@sun", password=")Ai_C40bj]6ssI9W", database="db_ideatrade"):
    with mysql.connector.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        database=database
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(query)
            columns = [column[0] for column in cursor.description]
            result = cursor.fetchall()

    return pd.DataFrame(columns=columns, data=result)

In [3]:
to_database_vm = query("SELECT * FROM allsymbol_global;")
to_database_vm.to_csv("to_database_vm.csv", index=False)

In [4]:
country_names_by_exchange = query("SELECT Country, Exchange FROM allsymbol_global ORDER BY `Country` desc")
# country_names_by_exchange.to_csv("country_names_by_exchange.csv", index=False)

In [5]:
# import pycountry
# from fuzzywuzzy import process
# from concurrent.futures import ThreadPoolExecutor

# # Function to get continent from country using fuzzy string matching
# def get_continent(country):
#     try:
#         # Use fuzzy string matching to find the closest matching country name
#         match, score = process.extractOne(country, [country.name for country in pycountry.countries])
        
#         # If the matching score is above a certain threshold, consider it a match
#         if score >= 90:
#             country_info = pycountry.countries.get(name=match)
#             if country_info:
#                 return country_info.continent.name
#         return None
#     except AttributeError:
#         return None

# # Function to apply get_continent in parallel
# def apply_get_continent_parallel(country_series):
#     with ThreadPoolExecutor() as executor:
#         return list(executor.map(get_continent, country_series))

# # Add a new column "Continent" to the DataFrame in parallel
# country_names_by_exchange['Continent'] = apply_get_continent_parallel(country_names_by_exchange['Country'])

# # Print the DataFrame with the new "Continent" column
# print(country_names_by_exchange)


In [6]:
count_c = 0
count_e = 0

# Assuming 'country' and 'exchange' are column names in country_names_by_exchange
for index, row in country_names_by_exchange.iterrows():
    if row['Country'] == "":
        count_c += 1
    if row['Exchange'] == "":
        count_e += 1

count_c, count_e


(4016, 0)

In [7]:
# ! pip install gspread google-auth google-auth-oauthlib google-auth-httplib2
# ! pip install --upgrade certifi
# ! pip install mysql-connector-python
# ! pip install google-auth

import pandas as pd
pd.options.display.float_format = "{:,.3f}".format

import numpy as np
import datetime
import mysql.connector
import yfinance as yf
import requests

import warnings
warnings.filterwarnings('ignore')
from functools import lru_cache

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'tahoma'
import matplotlib.ticker as ticker

import gspread
from google.oauth2.service_account import Credentials
#import gspread_dataframe as gd

In [8]:
from datetime import datetime, timedelta
def get_data():
    # โหลดข้อมูลรับรอง JSON จากไฟล์ที่คุณดาวน์โหลด
    credentials = Credentials.from_service_account_file('keys.json', scopes=['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])

    # ตั้งค่าการเชื่อมต่อ gspread ด้วยข้อมูลรับรอง
    gc = gspread.service_account(filename='keys.json')

    # เปิดสเปรดชีตของ Google Sheets โดยใช้ชื่อหรือ URL
    spreadsheet = gc.open('stores_shopee')
 
    # เลือกชีตที่ต้องการใน Google Sheets
    worksheet = spreadsheet.worksheet('ชีต2')

    # อ่านข้อมูลจากชีตเป็นรายการของดิกชันนารี
    data = worksheet.get_all_records()

    # สร้าง DataFrame จากข้อมูล
    df = pd.DataFrame(data)

    return df
df = get_data()

In [9]:
def get_world_data():
    # โหลดข้อมูลรับรอง JSON จากไฟล์ที่คุณดาวน์โหลด
    credentials = Credentials.from_service_account_file('keys.json', scopes=['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])

    # ตั้งค่าการเชื่อมต่อ gspread ด้วยข้อมูลรับรอง
    gc = gspread.service_account(filename='keys.json')

    # เปิดสเปรดชีตของ Google Sheets โดยใช้ชื่อหรือ URL
    spreadsheet = gc.open('stores_shopee')
 
    # เลือกชีตที่ต้องการใน Google Sheets
    worksheet = spreadsheet.worksheet('world_counties')

    # อ่านข้อมูลจากชีตเป็นรายการของดิกชันนารี
    data = worksheet.get_all_records()

    # สร้าง DataFrame จากข้อมูล
    df = pd.DataFrame(data)

    return df
dfw = get_world_data()

In [12]:
for i in dfw[['Country', 'Continent']].iterrows():
    for j in df[['Country']].iterrows():
        if j[1]['Country'] == i[1]['Country']:
            df.loc[j[0], 'Continent'] = i[1]['Continent']


In [7]:

vm = 1
df_vm = df[df['vm'] == vm]
df_vm.reset_index(drop=True, inplace=True)



In [8]:
df_vm

,id,Symbol,full_name,VWAP,Price,Exchange,Country,Sector,vm
0,3,ZZZD,BMO TATICAL DIVIDEND ETF FUND,28.000,28.000,NEO,Canada,Miscellaneous,1
1,7,ZZG,INFORMATION SVCS GROUP INC,0.000,3.980,SWB,United States,Commercial Services,1
2,11,ZYXI,"Zynex, Inc.",8.720,8.650,NASDAQ,United States,Health Technology,1
3,15,ZYRX,ZYREXINDO MANDIRI BUANA TBK,177.330,179.000,IDX,Indonesia,Electronic Technology,1
4,19,ZYDUSLIFE,ZYDUS LIFESCIENCES,628.770,631.850,NSE,India,Health Technology,1
...,...,...,...,...,...,...,...,...,...
26460,105843,25,SHN TELLUS HLDNGS,16.850,16.900,SZSE,China,Finance,1
26461,105847,20,S/Z ZHONGHENG HUAF,12.630,12.570,SZSE,China,Electronic Technology,1
26462,105851,14,SHAHE INDUSTRIAL C,11.590,11.640,SZSE,China,Finance,1
26463,105855,9,CHINA BAOAN GROUP,11.480,11.460,SZSE,China,Producer Manufacturing,1


In [10]:
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
from tqdm import tqdm

tv = TvDatafeed()

def get_historical_data(tv, symbol, exchange):
    try:
        # Convert numeric symbol to string
        if isinstance(symbol, (int, float)):
            symbol = str(symbol)

        # Check if symbol and exchange are strings before using strip()
        if isinstance(symbol, str) and isinstance(exchange, str):
            symbol = symbol.strip()
            exchange = exchange.strip()

            intervals = [
                Interval.in_1_minute, Interval.in_3_minute, Interval.in_5_minute,
                Interval.in_15_minute, Interval.in_30_minute, Interval.in_45_minute,
                Interval.in_1_hour, Interval.in_2_hour, Interval.in_3_hour,
                Interval.in_4_hour, Interval.in_daily, Interval.in_weekly, Interval.in_monthly
            ]

            data = [tv.get_hist(symbol=symbol, exchange=exchange, interval=interval,
                                n_bars=10000 if 'daily' in str(interval) else 6000) for interval in intervals]

            return pd.concat(data)

        else:
            print(f"Skipping {symbol} on {exchange}: Symbol and exchange must be strings.")
            return pd.DataFrame()  # Return an empty DataFrame for symbols that couldn't be retrieved
    except Exception as e:
        print(f"Error retrieving data for {symbol} on {exchange}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame for the symbol that couldn't be retrieved

# Execute the query to get symbols and exchanges
sym_ex = df_vm[['Symbol', 'Exchange']]

# Iterate through symbols and exchanges, and fetch data for each combination
all_data = []
s = 0
for ind, row in tqdm(sym_ex.iterrows(), "Getting TV"):
    symbol = row['Symbol']
    exchange = row['Exchange']
    data = get_historical_data(tv, symbol, exchange)
    s += 1
    if not data.empty:
        all_data.append(data)
    if s == 2:
        break

# Concatenate the data for all symbols and exchanges
result = pd.concat(all_data)

# Now, 'result' contains the concatenated historical data for all symbols in every exchange
# You can proceed to use or store this data as needed


you are using nologin method, data you access may be limited
Getting TV: 0it [00:00, ?it/s]

Getting TV: 1it [00:22, 22.43s/it]


In [9]:

# result[result['symbol'] == 'NEO:ZZZ']

In [10]:
print(result[result['symbol'] == 'NEO:ZZZ'])

                      symbol   open   high    low  close      volume
datetime                                                            
2022-07-11 20:44:00  NEO:ZZZ 24.820 24.820 24.820 24.820     100.000
2022-07-11 22:23:00  NEO:ZZZ 24.740 24.740 24.740 24.740     100.000
2022-07-11 22:28:00  NEO:ZZZ 24.720 24.720 24.720 24.720     200.000
2022-07-11 22:48:00  NEO:ZZZ 24.750 24.750 24.750 24.750     100.000
2022-07-11 23:32:00  NEO:ZZZ 24.830 24.830 24.830 24.830     100.000
...                      ...    ...    ...    ...    ...         ...
2023-08-01 20:30:00  NEO:ZZZ 28.790 28.800 23.350 24.700 276,197.000
2023-09-01 20:30:00  NEO:ZZZ 24.180 24.180 22.210 23.270 103,220.000
2023-10-02 20:30:00  NEO:ZZZ 23.190 23.190 21.360 21.860 178,866.000
2023-11-01 20:30:00  NEO:ZZZ 21.960 24.490 21.600 23.910  62,165.000
2023-12-01 21:30:00  NEO:ZZZ 24.040 26.590 24.000 25.800  42,325.000

[53029 rows x 6 columns]
